# Lösen der 1-D Diffusionsgleichung (implizit)

In [1]:
using Plots, ExtendableSparse, LinearAlgebra, Printf
using GeoModBox.HeatEquation.OneD

Wenden wir nun das implizite finite Differenzen Schema auf unser 1-D Problem der Wärmediffusionsgleichung an. Zur Wiederholung hier noch einmal das Problem:

## Das Problem

Wir sind an der Änderung der Temperatur T mit der Zeit t interessiert, für eine bestimmte Anfangstemperatur T0(x,t0) und Randbedingungen; z.B. eine Dikeintrusion in der Lithosphäre (mit *L* = 100 m; *W* = 5 m; *κ* = 10⁻⁶ m²/s).

<img src="./Figures/Exercise02_1.png" alt="drawing" width="350"/> <br>
**Figure 1.** Sketch des geologischen Problem und des Profiles der Anfangstemperaturbedingung. 

Wie lange würde es dauern, bis sich der Dike auf eine bestimmte Temperatur abgekühlt hat?

Zur Bestimmung der Dauer der Abkühlung müssen wir die Diffusionsgleichung der Temperatur lösen (parabolische PDG):

$$ 
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right), 
$$

wobei *k* die thermische Leitfähigkeit, *T* die Temperatur und $\rho$ die Dichte ist. Durch Umformen (Annahme, dass die thermischen parameter konstant sind!) können wir die Gleichung durch die thermische Diffusivität $\kappa = \frac{k}{\rho c_p}$ ausdrücken: 

$$
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}.
$$

### Finite Differenzen Approximation

Zur numerischen Lösung der Problems mit Hilfe von finiten Differenzen müssen wir zuerst ein numerisches Gitter erstellen (die Diskretisierung):

<img src="./Figures/Exercise02_2.png" alt="drawing" width="350"/> <br>
**Figure 2.** Numerisches 1-D Gitter für die Diskretisierung der *PDG*. **Must be corrected for the central nodes!!!**

Um die Diffusionsgleichung numerisch mit Hilfe des impliziten finiten Differenzen Schema zu lösen müssen wir
die PDG wie folgt umformulieren:

$$
\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t} = \frac{T_{i+1}^{n+1} - 2T_{i}^{n+1} + T_{i-1}^{n+1}}{(\Delta x)^2}
$$

Diese Gleichung können wir umformulieren, so dass wir ein Gleichungssystem erhalten, mit sovielen
Gleichungen wie wir zentral Gitterpunkte haben:

$$
-a T_{i+1}^{n+1} + \left(2a + b\right) T_{i}^{n+1} - a T_{i-1}^{n+1} = b T_{i}^{n}, 
$$

mit 
$$
a=\frac{\kappa}{\Delta{x^2}}, b = \frac{1}{\Delta{t}}.
$$

D.h. wir haben ein tridiagonales (drei Diagonalen) Gleichungssystem, welches durch eine Koeffizientenmatrix $\bm{A}$, einem unbekannten Vektor $T^{n+1}$ und einen bekannten Vektor $T^n$ beschrieben werden kann. 

### Parameterdefinitionen

Definieren wir für das Problem erst einmal bestimmte Parameter (physikalische Konstanten, numerische Domaine, Zeitparameters):

In [ ]:
alternative =   1

# Physikalische Parameter ----------------------------------------------- #
L           =   ?   # Laenge der Modeldomain [m]
Tdike       =   ?  # Temperatur des Dikes [C]
Trock       =   ?   # Temperatur des Umgebungsgesteins [C]
κ           =   ?  # Thermische Diffusivitaet des Gesteins [m2/s]
W           =   ?     # Breite des Dykes [m]
# ----------------------------------------------------------------------- #
# Numerische Parameter -------------------------------------------------- #
nc          =   ?                 # Anzahl der Gitterpunkte in x-Richtung
Δx          =   ?                # Gitterlaenge
xc          =   ?   # Gitter
# Iterations --  defection correction method
niter       =   10  
ϵ           =   1.0e-10       
R           =   zeros(nc)   # Residual
# ----------------------------------------------------------------------- #
# Zeit Parameter -------------------------------------------------------- #
day         =   3600.0*24.0     # Sekunden pro Tag
fac         =   1.0 
Δt          =   ?
tmax        =   365.0*day 
nt          =   ceil(Int,tmax/Δt)
time        =   0.0
# ----------------------------------------------------------------------- #

### Anfangsbedingungen

Jetzt wollen wir erst einmal die Anfangsbedingung graphisch darstellen. 

In [ ]:
# Anfangsbedingungen; Temperaturprofil ---------------------------------- #
T           =   ?       # Temperatur des Umgebungsgesteins
@. T[?] =  ?           # Temperatur des Dikes
T_ex        =   ?
∂T2∂x2      =   ?
# ----------------------------------------------------------------------- #
# Plot initial condition ------------------------------------------------ #
p = plot( ? )

display(p)
# ----------------------------------------------------------------------- #

### Randbedingungen

Zur Lösung unseres Problem, müssen wir noch die Anfangsbedingungen definieren. Dazu nehmen wir 300 °C für das Umgebungsgestein und 1200 °C für den Dike an. D.h. die Anfangstemperatur ist definiert durch:

$$
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300, 
$$
$$
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200.
$$

Da wir zentrale Gitterpunkte für die Temperatur verwenden, liegt kein Gitterpunkt direkt auf den Rändern (hingegen jeder Erwartung ist das allerdings ein Vorteil!). Zur Festlegung der Temperaturrandbedingungen müssen wir uns also der zusätzlichen *Ghost nodes* bediehnen (siehe Abbildung 2), d.h. wir bestimmen die Temperatur auf den *Ghostnodes* um die partielle Differentialgleichung der Temperatur auf dem **nächsten inneren** Gitterpunkt mit Hilfe der finiten Differenzen lösen zu können. Für eine konstante Temperaturbedingung and den Rändern (Dirichlet), können wir die Temperatur der *Ghostnodes* durch lineare Interpolation bestimmem, so dass: 

**West**
$$
T_{Ghost,W} = 2 T_{BC,W} + T[1],
$$
**East**
$$
T_{Ghost,E} = 2 T_{BC,E} + T[nc].
$$

Da wir ein tridiagonales Gleichungssystem haben, muss man die Koeffizienten und die rechte Seite für die Gleichungen des **ersten** und **letzten** **inneren Gitterpunktes** in Abhängigkeit der Randbedingungen wie folgt modifizieren (Herleitung siehe Vorlesung): 

#### **Dirichlet**
*West*
$$
- a T_{2}^{n+1} + \left(3 a + b \right) T_{1}^{n+1} = b T{1}^{n} + 2 a T_{BC,W}
$$
*East*
$$
\left(3 a + b \right) T_{nc}^{n+1} - a T_{nc-1}^{n+1}= b T_{nc}^{n} + 2 a T_{BC,E}
$$

#### **Neumann**
*West*
$$
- a T_{2}^{n+1} + \left(a + b \right) T_{1}^{n+1} = b T_{1}^n - a c_{W} \Delta{x}
$$
*East*
$$
\left(a + b \right) T_{nc}^{n+1} - a T_{nc-1}^{n+1} = b T_{nc}^n - a c_{E} \Delta{x}
$$

Bei der Initialisierung der Randbedingungen bediehnung wir uns eines Tricks und definieren die Temperatur auf den *Ghostnodes* später im Skript: 

In [ ]:
# Randbdingungen -------------------------------------------------------- #
BC   = (
    type = (W=:Dirichlet, E=:Dirichlet),
    # type = (W=:Neumann, E=:Neumann),
    val  = (W=300.0, E=300.0))
#end
# ----------------------------------------------------------------------- #

### Gleichungssystem

Lasst uns nun die Koeffizientenmatrix und rechte Seite initialisieren: 

In [ ]:
# Assemble Coefficient Matrix ------------------------------------------- #
# Definition der Matrix (hier in Sparse Form)
ndof        =   ?
K           =   ExtendableSparseMatrix(ndof,ndof)    
# ----------------------------------------------------------------------- #
# Setup right hand side ------------------------------------------------- #
Told            =   ?
Told[2:nc-1]    =   ?
Told[1]         =   ?
Told[nc]        =   ?
# ----------------------------------------------------------------------- #

### Visualisierung

Zur Visualisierung der Ergebnisse als eine Animation in einer GIF Datei, müssen noch der Ort und der Name der Datei festgelegt werden: 

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("03_1D_implicit_",alternative)
save_fig    =   0
# ----------------------------------------------------------------------- #          

### Zeitschleife

Numerisch können wir nun in einer Zeitschleife die *PDG* auf unterschiedliche Art und Weise lösen (Alternative I ist ausreichend; wer möchte, kann auch die Alternativen II und III lösen). 

In [ ]:
# Timestep loop --------------------------------------------------------- #
for n = 1:nt
    println("Zeitschritt: ",n,", Time: $(round(time/day, digits=1)) [d]")
    if alternative == 1
        a   =   ?
        b   =   ?

        @. Told       =   ?

        # Alternative II
        for i = 1:nc  
            # Equation number
            ii          =   i
            # Stencil 
            iW          =   ii - 1
            iC          =   ii
            iE          =   ii + 1   
            # Boundaries 
            inW    =  i==1    ? false  : true
            DirW   = (i==1    && BC.type.W==:Dirichlet) ? 1. : 0.
            NeuW   = (i==1    && BC.type.W==:Neumann  ) ? 1. : 0.
            inE    =  i==nc ? false  : true
            DirE   = (i==nc && BC.type.E==:Dirichlet) ? 1. : 0.
            NeuE   = (i==nc && BC.type.E==:Neumann  ) ? 1. : 0.
            if inE
                K[ii,iE]    = ?
            end
            K[ii,iC]        =  ?
            if inW 
                K[ii,iW]    = ?
            end        
        
            # Aenderung der rechten Seite durch die Randbedingungen ------------- #        
            if inE 
                # West boundary
                Told[1]   = ?
            end
            if inW
                # East boundary
                Told[nc]  = ?
            end
        end
            
        T       =   K\Told
    elseif alternative == 2            
        T, K    =   BackwardEuler!(?)
    elseif alternative == 3
        for iter = 1:niter
            # Residual iteration
            ComputeResiduals!(?)
            @printf("||R|| = %1.4e\n", norm(R)/length(R))            
            norm(R)/length(R) < ϵ ? break : nothing
            # Assemble linear system
            K  = AssembleMatrix(?)
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = ?             
            # Update temperature            
            T = T .+ δT            
        end        
    end
    # Überschreiben wir nur das alte Temperaturfeld mit dem neuen      
    @. Told    =   ? 
    # Berechnung der Zeit ---
    time    =   time + Δt        
    # Plot Lösung ---
    p = plot( ? )
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end
end

Nun müssen wir noch die Animation erstellen und speichern:

In [ ]:
# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(endswith(".png"), readdir(path,join=true)))
# ----------------------------------------------------------------------- #